In [3]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
from IPython.display import Markdown,display
import datetime
import time

df = pd.read_csv('data/DATA_20171130.csv')

In [54]:
"""
function: select asset that contains fuel consumption information
"""
def select_subset(df,num_asset,keys,file_path1,file_path2):
    keys_fuel = ['MDI_DASHBOARD_FUEL','MDI_OBD_FUEL']
    keys_distance = ['ODO_FULL_METER','MDI_OBD_MILEAGE','MDI_DASHBOARD_MILEAGE']
    for i in range(num_asset):
        asset = df.loc[df['asset'] == i]
        #choose the asset that contains the information of gas consumption
        if len(    asset.loc[  ( ( (asset[keys_fuel[0]]!=' ') & (asset[keys_fuel[0]]!='0') ) 
                                |( (asset[keys_fuel[1]]!=' ') & (asset[keys_fuel[1]]!='0') ) 
                               )
                             & ( ( (asset[keys_distance[0]]!=' ') & (asset[keys_distance[0]]!='0') ) 
                                |( (asset[keys_distance[1]]!=' ') & (asset[keys_distance[1]]!='0') ) 
                                |( (asset[keys_distance[2]]!=' ') & (asset[keys_distance[2]]!='0') ) )] ) >0:
            file_name = str(i)+'.csv'  
            asset.to_csv(file_path1+file_name)           #original data
            asset[keys].to_csv(file_path2+file_name)     #data contains key info

#keys : import informations 
keys = ['asset','recorded_at','MDI_DASHBOARD_FUEL','MDI_OBD_FUEL','ODO_FULL_METER','MDI_OBD_MILEAGE','MDI_DASHBOARD_MILEAGE']
file_path1 = 'data/data_by_asset/'
file_path2 = 'data/data_by_asset_keys/'
#in reality many of the asset is empty. Only 5000 assets contain valide data
num_asset = 5000
select_subset(df,num_asset,keys,file_path1,file_path2)      

In [4]:
"""
function : divide a asset into small pieces according to the time interval
"""
def divise_asset_by_time(file_full_path):
    df_asset = pd.read_csv(file_full_path)
    df_list = []
    count_data = len(df_asset)
    
    start_line = 0
    for i in range(1,count_data):
        last_time      = time.mktime( datetime.datetime.strptime(df_asset.iloc[i-1]['recorded_at'], "%Y-%m-%dT%H:%M:%SZ").timetuple() )
        current_time   = time.mktime( datetime.datetime.strptime(df_asset.iloc[i]['recorded_at'], "%Y-%m-%dT%H:%M:%SZ").timetuple() )
        
        #here we consider the interval between 2 journey is greater than 30 minutes
        interval = 1800
        if current_time-last_time > interval:
            df_list.append(df_asset.iloc[start_line:i])
            start_line = i     
        if i == count_data-1:    
            df_list.append(df_asset.iloc[start_line:count_data])
    return df_list

file_path = 'data/data_by_asset_keys/'
file_name = '4.csv'
df_list = divise_asset_by_time(file_path+file_name)

for i in range(len(df_list)):
    print("start time :",df_list[i].iloc[0]['recorded_at'], " end time :",df_list[i].iloc[-1]['recorded_at'])
